In [ ]:
import numpy as np
import pandas as pd

# 5.1. Tratamiento de series temporales I.

Trabajar con fechas y con series en el tiempo es muy intuitivo.

## Dates y times nativos de Python: ``datetime`` and ``dateutil``

Los objetos básicos de Python para trabajar con fechas y horas se encuentran en el módulo ``datetime``.

Puedes construir una fecha manualmente usando el tipo ``datetime``

In [ ]:
from datetime import datetime

Definimos una fecha con datetime

In [ ]:
datetime(year=2015, month=7, day=4)

Obtenemos el día de hoy

In [ ]:
now = datetime.now()
now

Podemos "interrogar" now por partes:

In [ ]:
print(now.year) 
print(now.month) 
print(now.day)

Si tenemos dos fechas, podemos calcular la diferencia entre ellas, para obtener el número de días

In [ ]:
delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)
delta

Podemos preguntar únicamente por la diferencia de días

In [ ]:
delta.days

O por la diferencia en segundos (no llega a 1 día, por eso no se suma a los días)

In [ ]:
delta.seconds

Podemos sumar y restar periodos con timedelta

In [ ]:
from datetime import timedelta

start = datetime(2011, 1, 7)
start + timedelta(12)

Básicamente, podemos realizar las operaciones que necesitemos

In [ ]:
start - 2 * timedelta(12)

Podemos convertir de Datime a Strings

In [ ]:
stamp = datetime(2011, 1, 3)

Con str obtenemos el string completo (con horas, minutos y segundos)

In [ ]:
str(stamp)

Con strftime, podemos indicar el formato de conversión de la fecha, a texto. En este caso, solo queremos año, mes y día.

In [ ]:
stamp.strftime('%Y-%m-%d')

También podemos hacerlo al revés. De texto a fecha.

In [ ]:
value = '2011-01-03'
datetime.strptime(value, '%Y-%m-%d')

Las fechas pueden venir con barras, con guiones... debemos indicar el formato a la hora de hacer la conversión

In [ ]:
datestrs = ['7/6/2011', '8/6/2011']

datetime.strptime(datestrs[0], '%m/%d/%Y')
# datetime.strptime(datestrs, '%m/%d/%Y') # No puedo pasar una lista, debería recorrerla con un for para hacer la transformación

Usando el módulo ``dateutil``, puedes parsear fechas de una gran variedad de formatos desde un string:

In [ ]:
from dateutil.parser import parse

Parse tiene gran variedad de formatos ya incluídos, por lo que es capaz de reconocer y convertir los string a fechas

In [ ]:
parse('2011-01-03')

In [ ]:
parse('Jan 31, 1997 10:45 PM')

In [ ]:
parse('6/12/2011', dayfirst=True)

El poder de  ``datetime`` y ``dateutil`` viene dado por su flexibilidad y fácil sintaxis.

Se pueden usar estos objetos y sus métodos para llevar a cabo casi cualquier operación. 

Su limitación viene al trabajar con vectores grandes: las listas de objetos datetime de Python son menos eficientes si las comparamos con pandas o numpy

## Pandas para time series

Pandas time series es lo que más se usa

### Estructuras de tiempo 

Uno de los usos más relevantes de Pandas en entorno financiero son las series de datos temporales. 

- Pandas tiene los objetos ``Timestamp`` que combinan la facilidad de  ``datetime`` y ``dateutil`` con una implementación vectorizada eficiente.
- Usando los objetos ``Timestamp``, pandas construye un ``DatetimeIndex``  para ser usado como índice en ``Series`` o ``DataFrame``.
- Las clases de Pandas para series temporales son:
     * ``Timestamp``: reemplazo de ``datetime``, basado en el sistema más eficiente de ``numpy.datetime64``. La estructura del índice asociada será ``DatetimeIndex``.
     * ``Period``: para periodos de tiempo. La estructura del índice asociado es ``PeriodIndex``.
     * ``Timedelta``: para variación de tiempo o duración (más eficiente que ``datetime.timedelta``). La estructura del índice asociado es ``TimedeltaIndex``.


- Los tipos más básicos son ``Timestamp`` y ``DatetimeIndex``.
- La forma más común de crear objetos de este tipo es usando la función ``pd.to_datetime()`` .
- Puede parsear una gran variedad de formatos. 
- Pasar una fecha por esta función produce un ``Timestamp``; pasar una serie de fechas por defecto devuelve un ``DatetimeIndex``:

Una de las funciones que más se usa es pd.to_datetime

Coge una columna que está como string y la pasa a fechas (la convierte en DatetimeIndex)

Veamos un ejemplo de su potencia

Cualquier ``DatetimeIndex`` puede convertirse en un ``PeriodIndex`` con la función ``to_period()``, añadiendo un código de frecuencia. Por ejemplo: ( ``'D'`` = diario) :

In [ ]:
dates.to_period('D')

In [ ]:
dates.to_period('M')

In [ ]:
dates.to_period('Y')

Un ``TimedeltaIndex`` se crea, por ejemplo, cuando restamos una fecha de otra:

In [ ]:
dates - dates[0]

Normalmente se pone como índice las fechas

Cuando recibimos un fichero, lo primero que hacemos es cambiar las fechas de string a fecha, y luego las asignamos como índice

In [ ]:
dates = [pd.Timestamp(2011, 1, 2), pd.Timestamp(2011, 1, 5),
         pd.Timestamp(2011, 1, 7), pd.Timestamp(2011, 1, 8),
         pd.Timestamp(2011, 1, 10), pd.Timestamp(2011, 1, 12)]

ts = pd.Series(np.random.randn(6), index=dates)
ts

Podemos consultar / extraer el índice con .index

In [ ]:
ts.index

### Indexación, seleccíon y subsección

Las herramientas de series temporales de Pandas se vuelven realmente útiles al utilizar *indexación*.

Generamos un rango de fechas y un DF, al que ponemos las fechas como índice, para poder hacer indexaciones. Este es el procedimiento más habitual.

In [ ]:
dates = pd.date_range('2015-07-25', periods=15, freq='B')

data = pd.DataFrame({'close':[10,12,14,15,15,19, 20,17, 15, 14, 12,13,13,14,10]}, 
                    index=dates)
data

Con el índice como DatetimeIndex se puede hacer uso de cualquier sistema de indexación.

Pasando valores que pueden ser convertidos en fechas:

Por ejemplo, podemos pasar dos fechas como un rango para extraer la información asociada entre ambas fechas (indexando por ese rango)

In [ ]:
data['2015-07-30':'2015-08-4']

Hay operaciones de indexación adicionales disponibles sólo para fechas.

Como pasar un año para obtener todos los datos asociados a ese año:

In [ ]:
data.loc['2015']

O a un año y un mes en concreto:

In [ ]:
data.loc['2015-07']

## Rangos de fechas: ``pd.date_range()``

- Igual que las funciones ``range()`` (nativo) y ``np.arange()`` (NumPy) que generan una secuencia tomando un punto de inicio, un punto de fin y un paso, las funciones ``pd.date_range()`` (para generar fechas entre un inicio y un fin) y ``pd.period_range()`` (para generar fechas entre un inicio, con un número de fechas a generar) funcionan de forma similar.


- Para crear un vector de fechas, se usa la función 'date_range'. El uso básico es:
     - `date_range(beginning_date, ending_date, freq)`, que devolvería una serie temporal diario entre ambas fechas (incluidas),
     - `date_range(beginning_date, periods=periods, freq)` que devolvería un rango de fechas desde beginning_date con tantas fechas como se especifiquen en 'periods'. 


In [ ]:
pd.date_range('2015-07-03', '2015-07-10')

COn date_range definimos el número de periodos (de fechas a generar)

In [ ]:
pd.date_range('2015-07-03', periods=8)

Se puede introducir un atributo ``freq`` para cambiar la frecuencia de las fechas de diaria (``D``) a otras diferentes.

Se dispone de las siguientes frecuencias:

| Code   | Description         |
|--------|---------------------|
| ``D``  | Calendar day        |
| ``B``  | Business day        |
| ``W``  | Weekly              |       
| ``M``  | Month end           | 
|``BM`` | Business month end   |
| ``Q``  | Quarter end         |
| ``BQ`` | Business quarter end|
| ``A``  | Year end            | 
|``BA`` | Business year end    |
| ``H``  | Hours               |
|``BH`` | Business hours       |
| ``T`` o ``min`` | Minutes             |       
| ``S``  | Seconds             |       
| ``L``  | Miliseconds         |       
| ``U``  | Microseconds        |     
| ``N``  | Nanoseconds         |       

Las frecuencias mensual, trimestral y anual se proporcionan como fecha el final del periodo especificado.

Añadiendo una 'S' al final de cada una de estas, la salida será al inicio del periodo.

| Code    | Description            |
|---------|------------------------|
| ``MS``  | Month start            |
| ``QS``  | Quarter start          |
| ``AS``  | Year start             |
|``BMS``  | Business month start   |
|``BQS``  | Business quarter start |
|``BAS``  | Business year start    |

Se podría cambiar el mes usado en frecuencias trimestrales, o anuales, añadiendo un código de tres letras al final:
 * ``Q-JAN``, ``BQ-FEB``, ``QS-MAR``, ``BQS-APR``, etc.
 * ``A-JAN``, ``BA-FEB``, ``AS-MAR``, ``BAS-APR``, etc.

En resumen, podemos construir los periodos que necesitemos, con tan solo una línea de código

Por ejemplo, dame 8 periodos de frecuencia horaria, dentro de un día determinado

In [ ]:
pd.date_range('2015-07-03', periods=8, freq='H')

O si queremos obtener todos los periodos entre dos fechas, por frecuencia horaria:

In [ ]:
pd.date_range('2015-07-03', '2015-07-04', freq='H')

Todo esto se puede combinar con números para otras frecuencias.

Por ejemplo, para frecuencias de 2 horas y 30 minutos, se puede combinar la hora ('H') y los minutos ('T')

Si queremos 8 periodos de frecuencia cada 2 horas y media:

In [ ]:
pd.date_range('2015-07-03', periods=8, freq='2H30min')

O para un día concreto de la semana del mes.

Como por ejemplo, el 3 viernes de cada mes:

In [ ]:
rng = pd.date_range('2012-01-01', '2012-09-01', freq='WOM-3FRI')
rng

Aunque no se suele usar, se puede usar ``pd.period_range()`` y ``pd.timedelta_range()`` para crear secuencias de ``Period`` o ``Timedelta``

Por ejemplo, para periodos mensuales:

In [ ]:
pd.period_range('2015-07', periods=8, freq='M')

O para secuencias de duración incremental de hora en hora:

In [ ]:
pd.timedelta_range(0, periods=10, freq='H')

Lo que se suele hacer es usar pd.date_range, para convertir las fechas de string a fecha, y convertirlas en el índice del DF

___
# Ejercicios

**5.1.1.** Genera un vector de fechas con 15 periodos de frecuencia mensual que empiece el 2015-1-1.

**5.1.2.** A continuación, genera 4 columnas de números aleatorios - que sigan una distribución normal estándar centrada en 10 y con una desviación típica de 2 - de la misma longitud que el vector de fechas. Redondea los números aleatorios a 6 decimales. 

**5.1.3.** Genera a partir de ambos inputs un DataFrame que tenga el vector de fechas como índice.

**5.1.4.** Accede a todos los elementos del año 2015, a todos los datos de abril del mismo año, y a los datos que se encuentran entre abril y septiembre del mismo año.

**5.1.5.** Convierte los siguientes strings en fechas que pandas pueda entender

- 07-07-2015
- 2015, 7, 3
- 4th of July, 2015
- 20150708
- 2015-Jul-6
- 12/11/1979

**5.1.6.** Construye un rango de fechas que empiece el 1 de enero de 2020, tenga 10 periodos con su frecuencia sea cada 9 días laborables.